In [1]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack
from preprocess import preprocess
import pandas as pd

In [2]:
# Load data
train = pd.read_csv('../data/challenge_1/train/classification_data.csv')
val = pd.read_csv('../data/challenge_1/val/classification_data.csv')

In [3]:
le = LabelEncoder()
train['responsible_entity_int'] = le.fit_transform(train['responsible_entity_id'])
val['responsible_entity_int'] = le.transform(val['responsible_entity_id'])


In [4]:
X_train_cat, X_train_text, tfidf = preprocess(train)

In [5]:
X_train_cat.head()

,0,1,2,3,4,5,6,7
0,0,0,1.0,0,0,0,0,0
1,0,1.0,0,0,0,0,0,0
2,0,0,0,0,0,0,1.0,0
3,0,0,0,0,0,0,1.0,0
4,0,0,0,0,0,1.0,0,0
